In [207]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

from sklearn.model_selection import train_test_split


import folium
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression


In [208]:
df=pd.read_csv("feature_eng_data.csv")

In [209]:
df.set_index('Index', inplace=True)

In [210]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [211]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,em_acount,country_id,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,1,0,29.0,0,35,0,87218.10,0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,35548.74,0,0,1
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,122179.11,0,0,1
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,0,50.0,0,22,0,119775.54,0,1,0
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,0,50.0,1,23,0,96796.23,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,50.0,1,22,0,43912.17,0,0,1
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,26.0,1,23,0,23334.99,0,0,1
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,1,0,50.0,0,47,0,96796.23,0,0,1


In [212]:
df['diff_productos'] = df.groupby('pk_cid')['Ahorro'].diff()

In [213]:
df['diff_productos']

Index
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
5962919    0.0
5962920    0.0
5962921    0.0
5962922    0.0
5962923    NaN
Name: diff_productos, Length: 5936633, dtype: float64

In [214]:
df['compra'] = (df['diff_productos'] >= 1).astype(int)

In [215]:
df['compra'].value_counts()

compra
0    5893633
1      43000
Name: count, dtype: int64

In [216]:
df[df['Ahorro'] > 1]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
742,1051788,2018-01-28,2015-08-14,KFC,1.0,2,0,0,0,0,...,37.0,1,39,0,184968.30,1,2,3,NaN,0
839,1052102,2018-01-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,2,2,NaN,0
1158,1051365,2018-01-28,2015-08-13,KFC,1.0,3,0,0,0,1,...,8.0,1,29,0,124656.36,2,3,3,NaN,0
2583,1056056,2018-01-28,2015-09-05,KFC,1.0,1,0,0,0,1,...,28.0,1,56,0,51062.61,3,2,2,NaN,0
2859,1055240,2018-01-28,2015-09-03,KAT,1.0,2,0,0,0,1,...,28.0,1,49,0,453427.35,0,2,2,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5954955,1158158,2019-05-28,2016-08-06,KFC,1.0,2,0,0,0,1,...,15.0,0,48,0,108019.92,0,2,1,0.0,0
5956541,1171165,2019-05-28,2016-08-29,KFC,1.0,3,0,0,0,1,...,28.0,1,32,0,160338.78,1,2,2,0.0,0
5957646,1173075,2019-05-28,2016-09-05,KAT,1.0,1,0,0,0,1,...,28.0,1,63,0,212865.30,0,2,2,0.0,0


In [217]:
#Comprobamos con un pk_cid que sabemso que tiene mas de un producto y vemos que detecta las compras correctamente

df[df['pk_cid'] == 1052102]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
839,1052102,2018-01-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,2,2,NaN,0
474177,1052102,2018-02-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,-1.0,0
695096,1052102,2018-03-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
969043,1052102,2018-04-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
981977,1052102,2018-05-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,2,2,1.0,1
1257658,1052102,2018-06-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,-1.0,0
1581945,1052102,2018-07-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
2047280,1052102,2018-08-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0
2327249,1052102,2018-09-28,2015-08-16,KAT,1.0,3,0,0,0,0,...,28.0,0,30,0,243343.77,0,1,1,0.0,0


In [218]:
#Llenamos los NaN del primer mes con '0' ya que no tenemos con que comparar 

In [219]:
df['diff_productos'].fillna(value=0, inplace=True)


In [220]:
df.isnull().any().value_counts()

False    32
Name: count, dtype: int64

In [221]:
#Por tanto 'compra' sera nuestro target para la predisposicion de compra de los productos "CUENTAS"

In [222]:
df['entry_date'] = pd.to_datetime(df['entry_date'])
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [223]:
df['Antiguedad'] = (df['pk_partition'] - df['entry_date']) / pd.Timedelta(days=30)


In [190]:
df['Antiguedad'] = df['Antiguedad'].round().astype(int)

In [224]:
df['mes'] = df['pk_partition'].dt.to_period('M')


In [225]:
serie_temporal = df.groupby('mes')['compra'].sum()


In [226]:
serie_temporal

mes
2018-01       0
2018-02    2433
2018-03    2305
2018-04    2264
2018-05    2026
2018-06    3125
2018-07    2942
2018-08    2171
2018-09    2836
2018-10    3036
2018-11    2668
2018-12    2987
2019-01    1827
2019-02    4807
2019-03    2492
2019-04    2287
2019-05    2794
Freq: M, Name: compra, dtype: int32

In [230]:
serie_temporal = serie_temporal.drop(serie_temporal.index[0])

In [233]:
#Intentaremos predecir cuantos productos de AHORRO venderemos el mes siguiente (JUNIO DE 2019)

df_regresion = serie_temporal.reset_index()
df_regresion['mes'] = df_regresion['mes'].astype(str)  # Convertimos el periodo a string para la regresión lineal

# Dividimos los datos en características (X) y la variable objetivo (y)
X = pd.to_numeric(df_regresion['mes'].str.replace('-', ''), errors='coerce').values.reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la regresión lineal
y = df_regresion['compra'].values

# Entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)

# Predecimos las compras del mes 06 del año 2019
mes_06_2019 = pd.to_numeric('2019-06'.replace('-', ''), errors='coerce').reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la predicción
prediccion_mes_06_2019 = modelo.predict(mes_06_2019)

print("Predicción de compras para el mes 06 del año 2019:", prediccion_mes_06_2019[0])


Predicción de compras para el mes 06 del año 2019: 2862.1481132075423


In [234]:
#Predicción de beneficios

Beneficios = prediccion_mes_06_2019[0]*40
Beneficios

#Beneficios estimados para JUNIO 2019 de cproductos de AHORRO = 114.485 € 

114485.9245283017

In [191]:
df = df.sort_values(by='pk_partition', ascending=False)


In [192]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad
Index,,,,,,,,,,,,,,,,,,,,,
5962923,1550586,2019-05-28,2019-05-13,KHE,1.0,3,0,0,0,0,...,0,37,0,126765.57,0,0,1,0.0,0,0
5667089,1490900,2019-05-28,2018-10-27,KHN,1.0,2,0,0,0,0,...,1,48,0,126765.57,0,1,2,0.0,0,7
5667091,1490855,2019-05-28,2018-10-27,KHQ,0.0,3,0,0,0,0,...,0,22,0,126765.57,0,0,1,0.0,0,7
5667092,1490854,2019-05-28,2018-10-27,KHQ,0.0,3,0,0,0,0,...,0,20,0,83582.13,0,0,1,0.0,0,7
5667093,1490824,2019-05-28,2018-10-27,KHQ,1.0,3,0,0,0,0,...,0,23,0,83582.13,0,0,1,0.0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159682,1329635,2018-01-28,2017-10-10,KHE,1.0,3,0,0,0,0,...,0,20,0,136075.68,0,0,1,0.0,0,4
159681,1329634,2018-01-28,2017-10-10,KHE,0.0,3,0,0,0,0,...,1,21,0,82753.14,0,0,1,0.0,0,4
159680,1329629,2018-01-28,2017-10-10,KFC,1.0,2,0,0,0,0,...,0,48,0,76751.52,1,0,1,0.0,0,4


In [193]:
df = df[df['pk_partition'] == '2019-05-28']


In [194]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad
Index,,,,,,,,,,,,,,,,,,,,,
5962923,1550586,2019-05-28,2019-05-13,KHE,1.0,3,0,0,0,0,...,0,37,0,126765.57,0,0,1,0.0,0,0
5667089,1490900,2019-05-28,2018-10-27,KHN,1.0,2,0,0,0,0,...,1,48,0,126765.57,0,1,2,0.0,0,7
5667091,1490855,2019-05-28,2018-10-27,KHQ,0.0,3,0,0,0,0,...,0,22,0,126765.57,0,0,1,0.0,0,7
5667092,1490854,2019-05-28,2018-10-27,KHQ,0.0,3,0,0,0,0,...,0,20,0,83582.13,0,0,1,0.0,0,7
5667093,1490824,2019-05-28,2018-10-27,KHQ,1.0,3,0,0,0,0,...,0,23,0,83582.13,0,0,1,0.0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815330,1089462,2019-05-28,2015-10-26,KHE,1.0,3,0,0,0,0,...,0,24,0,157631.79,0,0,1,0.0,0,44
5815334,1089459,2019-05-28,2015-10-26,KHE,0.0,3,0,0,0,0,...,0,25,0,71066.58,0,0,1,0.0,0,44
5815331,1089461,2019-05-28,2015-10-26,KHE,0.0,2,0,0,0,0,...,0,64,0,117510.09,0,0,1,0.0,0,44


In [195]:
df = df.drop(['entry_date', 'pk_partition'], axis=1)


In [196]:
df = df.drop(['deceased'], axis=1)

In [197]:
df = df.drop(['pk_cid'],axis=1)

In [198]:
df = df.drop(['diff_productos'],axis=1)

In [199]:
df= pd.get_dummies(df, columns=['entry_channel'], drop_first=True)

In [200]:
X = df.drop('compra', axis=1)  # Features
y = df['compra']  # Target

In [201]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [202]:
print("Tamaño del conjunto de entrenamiento (X):", X_train.shape)
print("Tamaño del conjunto de prueba (X):", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

Tamaño del conjunto de entrenamiento (X): (352244, 38)
Tamaño del conjunto de prueba (X): (88061, 38)
Tamaño del conjunto de entrenamiento (y): (352244,)
Tamaño del conjunto de prueba (y): (88061,)


In [203]:
def AUC(model,X,y):
 return roc_auc_score(y,model.predict_proba(X)[:,1])
def cross_validation(model,X,y):
 Kfold_metric = cross_val_score(model, X, y, cv=4, scoring='roc_auc')
 return Kfold_metric.mean()

In [204]:
label_ = []
metric_train_ = []
metric_valida_ = []

In [205]:
def evaluate_model(label, model, X, y):
 model.fit(X,y)
 metric_train = AUC(model,X,y)
 metric_valida = cross_validation(model,X,y)
 
 label_.append(label)
 metric_train_.append(metric_train)
 metric_valida_.append(metric_valida)
 return pd.DataFrame({'Model': label_,
                'AUC Train': metric_train_,
                'AUC Cross-Valida': metric_valida_
                }).sort_values('AUC Cross-Valida')

In [206]:
evaluate_model(label='XGBoost', 
 model = xgb.XGBClassifier(), 
 X=X_train, y=y_train)

,Model,AUC Train,AUC Cross-Valida
0,XGBoost,0.997005,0.988567
